In [24]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD

In [25]:
train_path = '../Datasets/train.csv'
test_path = '../Datasets/test.csv'

In [26]:
train_df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=300)

In [27]:
train_df = pd.read_csv(train_path)
train_df = train_df.drop(['id', 'target'], axis = 1)

In [28]:
X = train_df.values

In [29]:
svd_100 = TruncatedSVD(n_components=100, n_iter=20, random_state=42)

In [30]:
svd_100.fit(X)

TruncatedSVD(algorithm='randomized', n_components=100, n_iter=20,
       random_state=42, tol=0.0)

In [31]:
print(svd_100.explained_variance_ratio_.sum()) 

0.7803804657811666


In [32]:
# we see that 100 columns explain upto 78% of variance. Let's try with 150 columns

In [33]:
svd_150 = TruncatedSVD(n_components=150, n_iter=20, random_state=42)
svd_150.fit(X)

TruncatedSVD(algorithm='randomized', n_components=150, n_iter=20,
       random_state=42, tol=0.0)

In [34]:
print(svd_150.explained_variance_ratio_.sum()) 

0.9246093979004004
